In [1]:
import os
os.chdir("/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project")
print(os.getcwd())

/Users/tessleggio/GoogleDrive/GeorgiaTech/2018-Fall/01-ISYE-6414/6414Project


In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandasql as ps
import glob
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [3]:
#AQI Data
data = pd.read_csv("data/daily_aqi_by_county_2014.csv")

for i in range(2015, 2019):
    data = pd.concat([data, pd.read_csv("data/daily_aqi_by_county_" + str(i) + ".zip")])    

In [4]:
data.head()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2014-01-02,33,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2014-01-05,29,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2014-01-08,29,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2014-01-11,15,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2014-01-14,22,Good,PM2.5,01-003-0010,1


In [5]:
#_______________________________________________________
#Loading in daily Air Pressure data for years 2014 - 2018

data_ap = pd.read_csv("data/daily_PRESS_2014.csv")

for i in range(2015, 2019):
    data_ap = pd.concat([data_ap, pd.read_csv("data/daily_PRESS_" + str(i) + ".zip")])
    
data_ap = data_ap[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]
data_ap.rename(columns={"Arithmetic Mean":"Average Air Pressure"}, inplace=True)

#Taking air pressure averages across sensor locations
data_ap = data_ap.groupby(["State Code", "County Code", "Date Local"])["Average Air Pressure"].mean().reset_index()

#_______________________________________________________
#Loading in daily Temperature data for years 2014 - 2018

data_t = pd.read_csv("data/daily_TEMP_2014.csv")

for i in range(2015, 2019):
    data_t = pd.concat([data_t, pd.read_csv("data/daily_TEMP_" + str(i) + ".zip")])
    
data_t = data_t[["State Name", "County Name","State Code","County Code", "Date Local", "Arithmetic Mean"]]
data_t.rename(columns={"Arithmetic Mean":"Average Temperature"}, inplace=True)

#Taking temperature averages across distinct sensor locations
data_t = data_t.groupby(["State Name", "County Name", "Date Local","State Code","County Code"])["Average Temperature"].mean().reset_index()


#_______________________________________________________________
#Loading in daily humidity/dew point data for years 2014 - 2018

data_h = pd.read_csv("data/daily_RH_DP_2014.csv")

for i in range(2015, 2019):
    data_h = pd.concat([data_h, pd.read_csv("data/daily_RH_DP_" + str(i) + ".zip")])

data_h = data_h[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]
data_h.rename(columns={"Arithmetic Mean":"Average Humidity"}, inplace=True)

#Taking humidity averages across distinct sensor locations
data_h = data_h.groupby(["State Code", "County Code", "Date Local"])["Average Humidity"].mean().reset_index()

    
#_______________________________________________________________
#Loading in daily windspeed point data for years 2014 - 2018
data_w = pd.read_csv("data/daily_WIND_2014.csv")

for i in range(2015, 2019):
    data_w = pd.concat([data_w, pd.read_csv("data/daily_WIND_" + str(i) + ".zip")])
    
data_w = data_w[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]
data_w.rename(columns={"Arithmetic Mean":"Average Windspeed"}, inplace=True)

#Taking Windspeed averages across distinct sensor locations
data_w = data_w.groupby(["State Code", "County Code", "Date Local"])["Average Windspeed"].mean().reset_index()

'#_______________________________________________________\n#Loading in daily Air Pressure data for years 2014 - 2018\n\ndata_ap = pd.read_csv("data/daily_PRESS_2014.csv")\n\nfor i in range(2015, 2019):\n    data_ap = pd.concat([data_ap, pd.read_csv("data/daily_PRESS_" + str(i) + ".zip")])\n    \ndata_ap = data_ap[["State Code", "County Code", "Date Local", "Arithmetic Mean"]]\ndata_ap.rename(columns={"Arithmetic Mean":"Average Air Pressure"}, inplace=True)\n\n#Taking air pressure averages across sensor locations\ndata_ap = data_ap.groupby(["State Code", "County Code", "Date Local"])["Average Air Pressure"].mean().reset_index()\n\n#_______________________________________________________\n#Loading in daily Temperature data for years 2014 - 2018\n\ndata_t = pd.read_csv("data/daily_TEMP_2014.csv")\n\nfor i in range(2015, 2019):\n    data_t = pd.concat([data_t, pd.read_csv("data/daily_TEMP_" + str(i) + ".zip")])\n    \ndata_t = data_t[["State Name", "County Name","State Code","County Code",

In [6]:
data.rename({'State Name':'State', 'county Name': 'County'}, axis=1, inplace=True)
data['State'] = data['State'].str.lower()
data['County'] = data['County'].str.lower()
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = pd.DatetimeIndex(data['Date']).year
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Day'] = pd.DatetimeIndex(data['Date']).day
data.head()

,State,County,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,Year,Month,Day
0,alabama,baldwin,1,3,2014-01-02,33,Good,PM2.5,01-003-0010,1,2014,1,2
1,alabama,baldwin,1,3,2014-01-05,29,Good,PM2.5,01-003-0010,1,2014,1,5
2,alabama,baldwin,1,3,2014-01-08,29,Good,PM2.5,01-003-0010,1,2014,1,8
3,alabama,baldwin,1,3,2014-01-11,15,Good,PM2.5,01-003-0010,1,2014,1,11
4,alabama,baldwin,1,3,2014-01-14,22,Good,PM2.5,01-003-0010,1,2014,1,14


In [7]:
data_t.rename({'State Name':'State', 'County Name': 'County'}, axis=1, inplace=True)
data_t['State'] = data_t['State'].str.lower()
data_t['County'] = data_t['County'].str.lower()
data_t['Date Local'] = pd.to_datetime(data_t['Date Local'])
data_t['Year'] = pd.DatetimeIndex(data_t['Date Local']).year
data_t['Month'] = pd.DatetimeIndex(data_t['Date Local']).month
data_t['Day'] = pd.DatetimeIndex(data_t['Date Local']).day
data_t.head()

data_t2 = data_t.groupby(['State', 'County','Year','Month','State Code','County Code'])['Average Temperature'].mean().reset_index()
data_t2.head()
#data_w = data_w.groupby(["State Code", "County Code", "Date Local"])["Average Windspeed"].mean().reset_index()

'\ndata_t.rename({\'State Name\':\'State\', \'County Name\': \'County\'}, axis=1, inplace=True)\ndata_t[\'State\'] = data_t[\'State\'].str.lower()\ndata_t[\'County\'] = data_t[\'County\'].str.lower()\ndata_t[\'Date Local\'] = pd.to_datetime(data_t[\'Date Local\'])\ndata_t[\'Year\'] = pd.DatetimeIndex(data_t[\'Date Local\']).year\ndata_t[\'Month\'] = pd.DatetimeIndex(data_t[\'Date Local\']).month\ndata_t[\'Day\'] = pd.DatetimeIndex(data_t[\'Date Local\']).day\ndata_t.head()\n\ndata_t2 = data_t.groupby([\'State\', \'County\',\'Year\',\'Month\',\'State Code\',\'County Code\'])[\'Average Temperature\'].mean().reset_index()\ndata_t2.head()\n#data_w = data_w.groupby(["State Code", "County Code", "Date Local"])["Average Windspeed"].mean().reset_index()\n'

In [8]:
#data_t2.to_csv("./cleaned_data/temps.csv", index=None, header=True)

In [9]:
query = '''SELECT * FROM data WHERE Year = 2016'''
data_2016 = ps.sqldf(query=query)
data_2016.to_csv("./cleaned_data/aqi_2016.csv", index=None, header=True)

In [10]:
query = """ SELECT State, County, MAX(AQI) as max_aqi, MIN(AQI) as min_aqi, AVG(AQI) as avg_aqi, Year, Month
            FROM data
            GROUP BY State, County, Year, Month
"""
data_agg = ps.sqldf(query=query)

In [11]:
data_agg.head()
print(len(data_agg))

49500


In [16]:
data.to_csv("./cleaned_data/aqi_all-years.csv", index=None, header=True)

In [17]:
display(data.head())
display(data_2016.head())

,State,County,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,Year,Month,Day
0,alabama,baldwin,1,3,2014-01-02,33,Good,PM2.5,01-003-0010,1,2014,1,2
1,alabama,baldwin,1,3,2014-01-05,29,Good,PM2.5,01-003-0010,1,2014,1,5
2,alabama,baldwin,1,3,2014-01-08,29,Good,PM2.5,01-003-0010,1,2014,1,8
3,alabama,baldwin,1,3,2014-01-11,15,Good,PM2.5,01-003-0010,1,2014,1,11
4,alabama,baldwin,1,3,2014-01-14,22,Good,PM2.5,01-003-0010,1,2014,1,14


,State,County,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,Year,Month,Day
0,alabama,baldwin,1,3,2016-01-01 00:00:00.000000,33,Good,PM2.5,01-003-0010,1,2016,1,1
1,alabama,baldwin,1,3,2016-01-04 00:00:00.000000,22,Good,PM2.5,01-003-0010,1,2016,1,4
2,alabama,baldwin,1,3,2016-01-07 00:00:00.000000,28,Good,PM2.5,01-003-0010,1,2016,1,7
3,alabama,baldwin,1,3,2016-01-10 00:00:00.000000,13,Good,PM2.5,01-003-0010,1,2016,1,10
4,alabama,baldwin,1,3,2016-01-13 00:00:00.000000,36,Good,PM2.5,01-003-0010,1,2016,1,13


In [13]:
#Checking Number of Counties for each State
#query = """SELECT 'State Code', COUNT('County Code') from data_t GROUP BY 'State Code'"""
#ps.sqldf(query=query)

In [14]:
'''data_t['Date Local'] = pd.to_datetime(data_t['Date Local'])
data_ap['Date Local'] = pd.to_datetime(data_ap['Date Local'])
data_h['Date Local'] = pd.to_datetime(data_h['Date Local'])
data_w['Date Local'] = pd.to_datetime(data_w['Date Local'])


#Merging AQI data with daily weather data
data = pd.merge(data, data_t, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])
data = pd.merge(data, data_ap, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])
data = pd.merge(data, data_h, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])
data = pd.merge(data, data_w, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])

#Extracting year and month from date
data["Year"] = pd.DatetimeIndex(data['Date']).year
data["Month"] = pd.DatetimeIndex(data['Date']).month

#Renaming Columns
data.rename(columns = {"State Name":"State", "county Name":"County"}, inplace=True)

display(data.head())
'''

'data_t[\'Date Local\'] = pd.to_datetime(data_t[\'Date Local\'])\ndata_ap[\'Date Local\'] = pd.to_datetime(data_ap[\'Date Local\'])\ndata_h[\'Date Local\'] = pd.to_datetime(data_h[\'Date Local\'])\ndata_w[\'Date Local\'] = pd.to_datetime(data_w[\'Date Local\'])\n\n\n#Merging AQI data with daily weather data\ndata = pd.merge(data, data_t, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])\ndata = pd.merge(data, data_ap, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])\ndata = pd.merge(data, data_h, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])\ndata = pd.merge(data, data_w, how="left", left_on=["State Code", "County Code", "Date"], right_on=["State Code", "County Code", "Date Local"])\n\n#Extracting year and month from date\ndata["Year"] = pd.DatetimeIndex(data[\'Date\']).year\ndata["Month"] = pd.

In [15]:
#Removing trailing blanks from county and state variables
#data["State"] = data["State"].apply(str.strip)
#data["County"] = data["County"].apply(str.strip)